In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)

Initialize a tokenizer instance

In [2]:
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

We need to set the normalization components, which are *convert to lowercase* -> *normalize with NFKC*

In [3]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.NFKD()]
)

Next we have pretokenization, eg how do we split into words before tokenization? For Dhivehi we want to split on both whitespace and punctuation (a comma isn't part of a word - it's separate). This is a common approach and covered with:

In [4]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

Next we train the tokenizer, we've already assigned the WordPiece tokenizer so all we do now is pass Dhivehi text data to the training function, specify any special tokens to include in our vocab (as they will not be found in the training data... Hopefully!), and specify our target vocab size.

In [5]:
trainer = trainers.WordPieceTrainer(
    vocab_size=30_000,
    special_tokens=['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]'],
    min_frequency=2,
    continuing_subword_prefix='##'
)

In [6]:
read_dv = open('../data/dv-corpus-clean-unique.txt', encoding='utf-8')
tokenizer.train_from_iterator(read_dv, trainer=trainer)
read_dv.close()

read_dv = open('../data/dv-corpus-clean-unique.txt', encoding='utf-8')
count = 0
with open('../data/dv-corpus-clean-unique-2m.txt', 'w', encoding='utf-8') as fp:
    for row in read_dv:
        fp.write(row+'\n')
        count += 1
        if count > 2_000_000: break
read_dv.close()

After training the tokenizer we need to set the post-processing steps, eg add any special tokens. At the start and end of every sequence we add a classifier `[CLS]` token and a seperator `[SEP]` token. We will be adding these using their token IDs which we can find with:

In [7]:
cls_id = tokenizer.token_to_id('[CLS]')
sep_id = tokenizer.token_to_id('[SEP]')

To set the post processing template we use something called the `TemplateProcessor`. This allows us to specify how to deal with both single sentences and sentence pairs (which we will need during NSP pretraining). The first/single sentence is represented by `$A` and the second (for pairs) is represented by `$B`.

Our special tokens, `$A`, and `$B` are all followed by an integer value which indicated their token type ID value.

In [8]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0',
    pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', cls_id),
        ('[SEP]', sep_id)
    ]
)

The final step is to set the `decoder` component of the tokenizer, here we just use the WordPiece tokenizer with the `##` word part prefix specified.

In [9]:
tokenizer.decoder = decoders.WordPiece(prefix='##')

Our tokenizer is now fully prepared, all that's left is to save it. To load the tokenizer with HF transformers we will first need to load our current tokenizer into a transformers fast tokenizer like so:

In [10]:
from transformers import PreTrainedTokenizerFast

full_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token='[UNK]',
    pad_token='[PAD]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    mask_token='[MASK]'
)

*(Fast tokenizers are faster than the usual tokenizers because they are implemented in Rust)*

In [11]:
full_tokenizer.save_pretrained('bert-base-dv')

('bert-base-dv\\tokenizer_config.json',
 'bert-base-dv\\special_tokens_map.json',
 'bert-base-dv\\tokenizer.json')

In [12]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('bert-base-dv')

In [13]:
tokenizer("(ސެންޓޭ)، ނާއިޒް ހަސަން (ދާދު) އަދި ހުސައިން ނިހާންގެ އިތުރުން ސަމްދޫހު މުހައްމަދު ހިމަނާފައިވެއެވެ")

{'input_ids': [2, 23, 15160, 8745, 5062, 2979, 4520, 23, 26862, 24, 2560, 5244, 20410, 3033, 11341, 2994, 9595, 3082, 23200, 2801, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

And we're done, our tokenizer is ready.